라이브러리 import

In [1]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

하이퍼 파라미터 정의

In [2]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

리플레이 버퍼 클래스 

In [3]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

Q 밸류 네트워크 클래스

In [4]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

학습 함수

In [5]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

메인 함수

In [6]:
def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

In [7]:
if __name__ == '__main__':
    main()

n_episode :20, score : 10.6, n_buffer : 212, eps : 7.9%
n_episode :40, score : 9.6, n_buffer : 403, eps : 7.8%
n_episode :60, score : 9.7, n_buffer : 596, eps : 7.7%
n_episode :80, score : 9.7, n_buffer : 790, eps : 7.6%
n_episode :100, score : 9.8, n_buffer : 987, eps : 7.5%
n_episode :120, score : 9.6, n_buffer : 1179, eps : 7.4%
n_episode :140, score : 9.4, n_buffer : 1367, eps : 7.3%
n_episode :160, score : 9.6, n_buffer : 1558, eps : 7.2%
n_episode :180, score : 10.3, n_buffer : 1765, eps : 7.1%
n_episode :200, score : 9.4, n_buffer : 1953, eps : 7.0%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)


n_episode :220, score : 10.3, n_buffer : 2160, eps : 6.9%
n_episode :240, score : 10.1, n_buffer : 2361, eps : 6.8%
n_episode :260, score : 9.9, n_buffer : 2559, eps : 6.7%
n_episode :280, score : 11.3, n_buffer : 2786, eps : 6.6%
n_episode :300, score : 23.9, n_buffer : 3263, eps : 6.5%
n_episode :320, score : 57.4, n_buffer : 4410, eps : 6.4%
n_episode :340, score : 131.2, n_buffer : 7033, eps : 6.3%
n_episode :360, score : 195.9, n_buffer : 10951, eps : 6.2%
n_episode :380, score : 167.8, n_buffer : 14308, eps : 6.1%
n_episode :400, score : 238.4, n_buffer : 19076, eps : 6.0%
n_episode :420, score : 162.6, n_buffer : 22328, eps : 5.9%
n_episode :440, score : 197.4, n_buffer : 26276, eps : 5.8%
n_episode :460, score : 235.9, n_buffer : 30995, eps : 5.7%
n_episode :480, score : 234.2, n_buffer : 35678, eps : 5.6%
n_episode :500, score : 379.6, n_buffer : 43269, eps : 5.5%
n_episode :520, score : 344.1, n_buffer : 50000, eps : 5.4%
n_episode :540, score : 307.1, n_buffer : 50000, eps :